In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import re

# Read data from file
file_path = "ballTouches.txt"
with open(file_path, "r") as file:
    logs = file.read()

# Parse the logs to extract behavior and locations
data = defaultdict(list)
pattern = r"'teamName': '([^']+)', 'location': '\(([^)]+)\)'"
matches = re.findall(pattern, logs)

# Sort matches by behavior name
matches.sort(key=lambda x: x[0])

# Separate behaviors based on color
purple_behaviors = [match for match in matches if match[0].endswith('(Purple)')]
blue_behaviors = [match for match in matches if match[0].endswith('(Blue)')]

# Function to process and plot heatmaps
def plot_heatmaps(behaviors_purple, behaviors_blue, title):
    data_purple = defaultdict(list)
    data_blue = defaultdict(list)
    
    for behavior, location in behaviors_purple:
        coords = tuple(map(float, location.split(", ")))
        data_purple[behavior].append(coords)
    
    for behavior, location in behaviors_blue:
        coords = tuple(map(float, location.split(", ")))
        data_blue[behavior].append(coords)
    
    n = max(len(data_purple), len(data_blue))
    fig, axes = plt.subplots(1, n, figsize=(20, 10), constrained_layout=True)
    
    for ax, ((behavior_blue, locations_blue), (behavior_purple, locations_purple)) in zip(axes[:], zip(data_blue.items(), data_purple.items())):
        locations_blue = np.array(locations_blue)
        locations_purple = np.array(locations_purple)
        x_blue, y_blue = locations_blue[:, 0], locations_blue[:, 2]  # Take x and z coordinates
        x_purple, y_purple = locations_purple[:, 0], locations_purple[:, 2]

        # merge blue and purple data
        x = np.concatenate((x_blue, x_purple))
        y = np.concatenate((y_blue, y_purple))

        heatmap, xedges, yedges = np.histogram2d(x, y, bins=[28, 60] , range=[[-7, 7], [-15, 15]])
        # heatmap_blue, xedges_blue, yedges_blue = np.histogram2d(x_blue, y_blue, bins=50, range=[[-7, 7], [-15, 15]])
        # heatmap_purple, xedges_purple, yedges_purple = np.histogram2d(x_purple, y_purple, bins=50, range=[[-7, 7], [-15, 15]])
        
        # Plot heatmap
        ax.imshow(heatmap.T, origin='lower', cmap='Blues', extent=[-7, 7, -15, 15])
        ax.set_title(behavior_blue.replace('(Blue)', '').strip(), fontsize=24)
        ax.tick_params(axis='both', which='major', labelsize=14)

        # plot lines to separate blue and purple heatmaps
        ax.axhline(0, color='black', linewidth=10)
    
    plt.suptitle(title, fontsize=24)
    plt.savefig("heatmaps.png")
    plt.show()

# Plot combined heatmaps for purple and blue behaviors
plot_heatmaps(purple_behaviors, blue_behaviors, '')
